<a href="https://colab.research.google.com/github/21Ovi/TensorFlow_/blob/main/06_NPL_in_TensorFlow_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in TensorFlow

NPL has the goal of deriving information out of natural language (could be sequences text to speech).

Another common term for NLP problems is sequence to sequence problem (seq2seq)

In [28]:
!nvidia-smi

Sun Jul 30 13:43:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    42W / 300W |    630MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [29]:
!wget https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py

# Import series of helper function for noteebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-07-30 13:43:12--  https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10116 (9.9K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]   9.88K  --.-KB/s    in 0s      

2023-07-30 13:43:12 (116 MB/s) - ‘helper_functions.py.1’ saved [10116/10116]



## Get a text dataset

The dataset we're going to using is Kaggle's introduction to NPL dataset (text samples of Tweets labelled as disaster or not disaster).

Resource : https://www.kaggle.com/competitions/nlp-getting-started

In [30]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-07-30 13:43:12--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  1.26MB/s    in 0.5s    

2023-07-30 13:43:13 (1.26 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in.

But I prefer to get visualize straigt away.

So another way to do this is to use pandas.

In [31]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [32]:
# Suffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [33]:
# What does the test dataframe look loke?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [34]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [35]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [36]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number of samples

for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text: 
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/mkWyvM3i8r

---

Target: 1 (real disaster)
Text: 
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/aZL4XydvzK

---

Target: 0 (not real disaster)
Text: 
@Hienshi @Gbay99 it wouldnt turn into a sandstorm if riot gave a good answer.  Instead they gave dumb excuses. At least new client is there

---

Target: 1 (real disaster)
Text: 
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/Z2Ph0ArzYI

---

Target: 0 (not real disaster)
Text: 
I liked a @YouTube video http://t.co/xR3xJJ8gJB Darude - Sandstorm

---



### Split data into training and validation sets

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
## Use train_train_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1, # use 10% of training data for validation set
                                                                             random_state=42)

In [39]:
# Check lengths
len(train_sentences), len(train_labels),len(val_sentences), len(val_labels),

(6851, 6851, 762, 762)

In [40]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are few ways to do this, namely:
* `Tokenization`- direct mapping of token ( a token could be a workd or a character) to number.
* `Embedding` - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [41]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [42]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=max_length, # how long do you want your sequence to be
                                    pad_to_max_tokens=True)

In [43]:
len(train_sentences[0].split())

7

In [44]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [45]:
# Setup text vactorize variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g how many words from a tweet does model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [46]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [47]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street~ "
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [48]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)

In [49]:
print(f"Orignal text: \n {random_sentence}\
      n\nVectorized Version:")
text_vectorizer([random_sentence])

Orignal text: 
 @david_hurn @ToKTeacher Why is there something in place to prevent skynet? Because perhaps there should be! http://t.co/73Umw2iGRZ      n
Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 7502,   91,    9,   74,  529,    4,  596,    5, 1378, 8441,
         152, 2806,   74,  168]])>

In [50]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represnted by a vector 100 long
* `input_length` = length of the sequence being passed to the embedding layer

In [51]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                            output_dim=128, # output shape
                             embeddings_initializer="uniform",
                            input_length=max_length # how long is each input
                            )

embedding

In [52]:
# get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Orignal text: \n {random_sentence}\
      \n\nEmbedding Version: ")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal text: 
 California man facing manslaughter charge in Sunday's wrong-way fatal crash in ... - http://t.co/1vz3RmjHy4: Ca... http://t.co/xevUEEfQBZ      

Embedding Version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00215769, -0.02588652, -0.04768885, ..., -0.02642143,
         -0.03262854, -0.00476541],
        [-0.01423331,  0.02637572, -0.04414188, ...,  0.04036567,
         -0.01248854, -0.03832476],
        [ 0.00424964,  0.03434621, -0.00209037, ..., -0.02823555,
          0.02852258,  0.04510696],
        ...,
        [ 0.03205308,  0.01054324,  0.00914944, ...,  0.02594215,
          0.00053345, -0.01174295],
        [ 0.04047245,  0.03908819,  0.01844413, ..., -0.01188971,
         -0.02826843,  0.0296502 ],
        [-0.04594679,  0.02624284,  0.01218529, ...,  0.01724714,
         -0.00163218, -0.03603927]]], dtype=float32)>

In [53]:
# Checkout a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00215769, -0.02588652, -0.04768885,  0.04862225, -0.03753135,
         0.01905764,  0.03452604,  0.01351203, -0.02106117,  0.00470568,
        -0.01869538, -0.04633741,  0.0495291 , -0.04935174, -0.03517378,
         0.00742744, -0.03099134,  0.0205498 , -0.01240136,  0.02733723,
        -0.00427768,  0.01418969,  0.01709816,  0.03107854, -0.01819489,
         0.03810097, -0.01100731,  0.04188773,  0.03370024,  0.02016877,
         0.02584367, -0.0481529 , -0.03651022, -0.01978929,  0.0284371 ,
         0.01018331, -0.03733108, -0.01501973,  0.01509321,  0.0315626 ,
         0.00248791, -0.04685878,  0.01259737,  0.03770551,  0.0144606 ,
         0.00965465,  0.00674882, -0.01931715,  0.03050881, -0.01905742,
         0.01185567,  0.03994756, -0.04855255, -0.0124132 , -0.04910235,
        -0.00776813,  0.03949405, -0.03219471, -0.03227028, -0.02570494,
        -0.02189793,  0.03963658, -0.02682505, -0.0317853 ,  0.0460952 ,
  

## Modelling a text dataset (running series of experiments)

No we've got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We've start with a baseline and move on from there

* `Model 0`: Naive Bayes (baseline), this is from SkLearn ML Map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* `Model 1`: Feed-forward neural network (dense model)
* `Model 2`: LSTM Model (RNN)
* `Model 3`: GRU Model (RNN)
* `Model 4`: Bidirectional-LSTM Model (RNN)
* `Model 5`: 1D Convolutional Neural Network (CNN)
* `Model 6`: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* `Model 7`: Same as model 6 with 10% of training data

How are we going to approch all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model


### Model 0: Getting a baseline

as with all machine learning modelling experiments, it's important to create  a baseline model so you've got b benchmark for future eperiments to build a model

To create a baseline, we'll yse SkLearn's multinominal Naive Bayes using the TensorFlow TF-IDF formula to convert our words to numbers

> 🔑 **Note:** It's common practice to use non-DL algorithm as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words into numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [56]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [58]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])